In [19]:
import pandas as pd
import json

pd.set_option('display.max_rows', 500)

import requests
from bs4 import BeautifulSoup

# Data Understanding

## Data Sources
* RKI, webscraping https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* John Hopkins (GIT) https://github.com/CSSEGISandData/COVID-19.git
* Rest API to retrieve covid data from NPGEO https://npgeo-corona-npgeo-de.hub.arcgis.com/


###  John Hopkins Source

In [3]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [4]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38054,38070,38113,38129,38140,38143,38162,38165,38196,38243
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,8605,8759,8927,9083,9195,9279,9380,9513,9606,9728
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,41858,42228,42619,43016,43403,43781,44146,44494,44833,45158
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,1060,1060,1098,1098,1124,1124,1124,1176,1184,1199
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,2222,2283,2332,2415,2471,2551,2624,2654,2729,2777


### Web Scraping from RKI


In [5]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

html_table = soup.find('table')
all_rows = html_table.find_all('tr')
print(all_rows)

[<tr><th colspan="1" rowspan="1">Bundes­land</th><th class="center" colspan="5" rowspan="1">Elektro­nisch über­mittelte Fälle</th></tr>, <tr><th colspan="1" rowspan="1"></th><th class="center" colspan="1" rowspan="1">An­zahl</th><th class="center" colspan="1" rowspan="1">Dif­fe­renz<br/>
zum <br/>
Vor­tag</th><th class="center" colspan="1" rowspan="1">Fälle<br/>
in den <br/>
letzten<br/>
7 Tagen</th><th class="right" colspan="1" rowspan="1">7-Tage-<br/>
Inzi­denz</th><th class="center" colspan="1" rowspan="1">Todes­fälle</th></tr>, <tr class="even"><td colspan="1" rowspan="1">Baden-Württem­berg</td><td class="right" colspan="1" rowspan="1">43.861</td><td class="right" colspan="1" rowspan="1">+269</td><td class="right" colspan="1" rowspan="1">1.552</td><td class="right" colspan="1" rowspan="1">14,0</td><td class="right" colspan="1" rowspan="1">1.867</td></tr>, <tr><td colspan="1" rowspan="1">Bayern</td><td class="right" colspan="1" rowspan="1">59.804</td><td class="right" colspan="1" ro

In [8]:
final_table_data = []
for pos,rows in enumerate(all_rows):
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_table_data.append(col_list)
print(final_table_data)

[[], [], ['Baden-Württem\xadberg', '43.861', '+269', '1.552', '14,0', '1.867'], ['Bayern', '59.804', '+506', '2.125', '16,3', '2.643'], ['Berlin', '11.792', '+65', '455', '12,1', '226'], ['Branden\xadburg', '3.943', '+12', '59', '2,3', '169'], ['Bremen', '2.073', '+3', '74', '10,8', '58'], ['Hamburg', '6.527', '+41', '200', '10,9', '267'], ['Hessen', '16.278', '+133', '669', '10,7', '535'], ['Meck\xadlenburg-Vor\xadpommern', '1.034', '+11', '26', '1,6', '20'], ['Nieder\xadsachsen', '17.372', '+88', '496', '6,2', '665'], ['Nord\xadrhein-West\xadfalen', '60.695', '+262', '1.617', '9,0', '1.823'], ['Rhein\xadland-Pfalz', '9.434', '+70', '300', '7,3', '243'], ['Saarland', '3.199', '0', '44', '4,4', '174'], ['Sachsen', '6.155', '+6', '126', '3,1', '225'], ['Sachsen-Anhalt', '2.287', '+5', '39', '1,8', '66'], ['Schles\xadwig-Holstein', '4.150', '+24', '103', '3,6', '161'], ['Thüringen', '3.694', '+4', '59', '2,8', '187'], ['Gesamt', '252.298', '+1.499', '7.944', '9,6', '9.329']]


In [16]:
pd.DataFrame(final_table_data).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatality',
                                                       5:'comment'}).head()

,state,cases,changes,cases_per_100k,fatality,comment
2,Baden-Württem­berg,43.861,+269,1.552,"14,0",1.867
3,Bayern,59.804,+506,2.125,"16,3",2.643
4,Berlin,11.792,+65,455,"12,1",226
5,Branden­burg,3.943,+12,59,"2,3",169
6,Bremen,2.073,+3,74,"10,8",58


## REST API CALLS

In [17]:
## data request for Germany
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [21]:
json_object=json.loads(data.content) 
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [22]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [24]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,4197,1599688800000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,144.888411,4.573731e+10,2.881496e+06,161
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,6626,1599688800000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,359.878100,2.089396e+09,4.188002e+05,267
2,3,03,Niedersachsen,Land,7982448,9,17572,1599688800000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,220.132972,1.299836e+11,4.008988e+06,665
3,4,04,Bremen,Freie Hansestadt,682986,5,2094,1599688800000,04,4132268b-54de-4327-ac1e-760e915112f1,306.594864,1.119157e+09,3.357177e+05,58
4,5,05,Nordrhein-Westfalen,Land,17932651,10,61201,1599688800000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,341.282502,8.782936e+10,2.648673e+06,1827


In [36]:
pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')
pd_full_list

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,4197,1599688800000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,144.888411,4.573731e+10,2.881496e+06,161
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,6626,1599688800000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,359.878100,2.089396e+09,4.188002e+05,267
2,3,03,Niedersachsen,Land,7982448,9,17572,1599688800000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,220.132972,1.299836e+11,4.008988e+06,665
3,4,04,Bremen,Freie Hansestadt,682986,5,2094,1599688800000,04,4132268b-54de-4327-ac1e-760e915112f1,306.594864,1.119157e+09,3.357177e+05,58
4,5,05,Nordrhein-Westfalen,Land,17932651,10,61201,1599688800000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,341.282502,8.782936e+10,2.648673e+06,1827
5,6,06,Hessen,Land,6265809,7,16500,1599688800000,06,93277ac4-e8fc-48c7-8940-028dc2ed66af,263.333913,5.235913e+10,2.148244e+06,537
6,7,07,Rheinland-Pfalz,Land,4084844,11,9560,1599688800000,07,e9b4296f-9be2-4e53-9a58-ccf1396cb03d,234.035865,4.783877e+10,1.774430e+06,245
7,8,08,Baden-Württemberg,Land,11069533,1,44396,1599688800000,08,80394ddf-c6a4-4a6e-be8e-0259a81b22a9,401.064797,8.151732e+10,2.544320e+06,1868
8,9,09,Bayern,Freistaat,13076721,2,60759,1599688800000,09,1ff920f4-62cd-4a4f-b8c9-f042f2a3e00a,464.634827,1.634855e+11,3.898618e+06,2644
9,10,10,Saarland,Land,990509,12,3218,1599688800000,10,e3396a6f-8a30-4fdf-8df7-def77dd38bea,324.883469,6.060692e+09,5.626789e+05,175


In [37]:
pd_full_list.shape[0]

16

## API access via REST service, e.g. USA data 


In [39]:
# US for full list
headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key': '28ee4219700f48718be78b057beb7eb4',
}

response = requests.get('https://api.smartable.ai/coronavirus/stats/US', headers=headers)
if response.status_code != 200:
    print("Something Wrong with Request!!")
else: 
    print("Request Success!")

Request Success!


In [43]:
US_dict=json.loads(response.content) # imports string
with open('../data/raw/SMARTABLE/US_data.json', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)

In [44]:
print(json.dumps(US_dict,indent=2))

ion": {
          "long": -114.742041,
          "countryOrRegion": "United States",
          "provinceOrState": "Idaho",
          "county": null,
          "isoCode": "US-ID",
          "lat": 44.068202
        },
        "totalConfirmedCases": 35795,
        "newlyConfirmedCases": 0,
        "totalDeaths": 424,
        "newDeaths": 0,
        "totalRecoveredCases": 10369,
        "newlyRecoveredCases": 0
      },
      {
        "location": {
          "long": -89.398528,
          "countryOrRegion": "United States",
          "provinceOrState": "Illinois",
          "county": null,
          "isoCode": "US-IL",
          "lat": 40.633125
        },
        "totalConfirmedCases": 256949,
        "newlyConfirmedCases": 0,
        "totalDeaths": 8466,
        "newDeaths": 0,
        "totalRecoveredCases": 146699,
        "newlyRecoveredCases": 0
      },
      {
        "location": {
          "long": -85.602364,
          "countryOrRegion": "United States",
          "provinceOrStat

In [56]:
full_list_US_country=[]
for pos,each_dict in enumerate (US_dict['stats']['breakdowns'][:]):
    flatten_dict=each_dict['location']
    flatten_dict.update(dict(list(US_dict['stats']['breakdowns'][pos].items())[1: 7]) 
    )
    full_list_US_country.append(flatten_dict)

In [68]:
pd.DataFrame(full_list_US_country).to_csv('../data/raw/SMARTABLE/full_list_US_country.csv',sep=';',index=False)